In [86]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import signal
import time
import datetime
import serial
from scipy.fftpack import fft,ifft
from matplotlib.pylab import mpl
import struct
import os
import serial
from pathlib import Path
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
!python.exe -m PyQt5.uic.pyuic Automated_testingV1.6.ui -o Automated_testingV16.py

In [22]:
!python.exe -m PyQt5.uic.pyuic Automated_testingV1.7.ui -o Automated_testingV17.py

In [229]:
!python.exe -m PyQt5.uic.pyuic Automated_testing.ui -o Automated_testing.py

Error: No such file or directory: "Automated_testing.ui"


In [ ]:
# 生成加密密钥
import uuid
import hashlib
mac = uuid.getnode()
uuids  = ':'.join(('%012X'%mac)[i:i+2] for i in range(0,12,2))
list_groupbox = [
    'groupBox_tools_bd3x_test',
    'groupBox_tools_bd3x_allpara',
    'comboBox_tools_bd3x_preset',
    'comboBox_tools_bd3x_preset'

]
print('本机UUID:{}'.format(uuids))
for i in list_groupbox:
    key = hashlib.md5(( i + uuids + 'yhc2025hashkey').encode("utf-8")).hexdigest()
    print('界面ID:{}, key:{}'.format(i,key))

本机UUID:40:EC:99:76:18:CF
界面ID:groupBox_tools_bd3x_test, key:f135c042e27556db9292f64b2e4b1b27
界面ID:groupBox_tools_bd3x_allpara, key:03f104489fdab0cef02309abbe2bf1aa
界面ID:comboBox_tools_bd3x_preset, key:030c56df3e9c56dd059db021c5f19175
界面ID:comboBox_tools_bd3x_preset, key:030c56df3e9c56dd059db021c5f19175


In [ ]:
def init_sdf():
    # 初始化文件
    list_columns = [
        "G0", "G+", "G-", 
        "ScaleFactor+", "ScaleFactor-", "NonLinear", 
        "K*x", "K*y", "K*z", 
        "Temp"
    ]
    list_rows = ["Gx", "Gy", "Gz", "Ax", "Ay", "Az"]
    sdf = pd.DataFrame(0,index=list_rows, columns=list_columns)
    return sdf  
def decode_zfG(zfG_str,A_length=4):
    zfG_list = zfG_str.split(' ')[:3]
    if len(zfG_list)<3:
        zfG_list = [zfG_str[:A_length]]*3
    zfG_decode = []
    for cout,strs in enumerate(zfG_list):
        if strs!=4:
            strs = strs.ljust(4,'0')[:4]
        zfg_list = [[],[]]
        for i in range(A_length):
            if strs[i]=='1':
                zfg_list[0].append(i)
            else:
                zfg_list[1].append(i)
        zfG_decode.append(zfg_list)

    return zfG_decode

    

def cal_paraSave_chy_LW(df,save_name, split_rule):
    g0_sort_rule = [0, 2, 1]
    a1_sort_rule = [1, 0, 2]
    zfG_encode = '1100 0101 1010'
    # G0所需计数 G1所需计数 GA偏移
    GA_shift = [4,4,4]
    zfG_decode = decode_zfG(zfG_encode,GA_shift[1])
    sdf = init_sdf()
    for i in range(3):
        sdf.iloc[g0_sort_rule[i],0] = df.iloc[ GA_shift[0]*i:GA_shift[0]*(i+1),i ].mean()
        sdf.iloc[g0_sort_rule[i]+3,0] = df.iloc[ GA_shift[0]*i:GA_shift[0]*(i+1),i+3 ].mean()
        sdf.iloc[a1_sort_rule[i],1] = np.mean([
            df.iloc[num+GA_shift[2]+GA_shift[1]*i,i] for num in zfG_decode[i][0]
        ])
        sdf.iloc[a1_sort_rule[i]+3,1] = np.mean([
            df.iloc[num+GA_shift[2]+GA_shift[1]*i,i+3] for num in zfG_decode[i][0]
        ])
        sdf.iloc[a1_sort_rule[i],2] = np.mean([
            df.iloc[num+GA_shift[2]+GA_shift[1]*i,i] for num in zfG_decode[i][1]
        ])
        sdf.iloc[a1_sort_rule[i],2] = np.mean([
            df.iloc[num+GA_shift[2]+GA_shift[1]*i,i] for num in zfG_decode[i][1]
        ])

        
    return sdf

    
paths = 'E:/Work/理工导航/General_autoTest_Host/test_fun/data/all_ave'
names = 'LW_-50_1_s_txt.txt'
para_name = names.split('.txt')[0] + '_para.txt'

df = pd.read_csv(os.path.join(paths, names), encoding='gb2312', delim_whitespace=True)
if df.columns[0].lower() == 'turntable':
    df = df.drop(df.columns[0], axis=1).reset_index(drop=True)
split_rule = ['LW','_',1,0]
# cal_paraSave_chy_LW(df, para_name, split_rule)
zfG_str = '1100 0101 1010   '
zfG_encode = decode_zfG(zfG_str)
print(zfG_encode)


[[[0, 1], [2, 3]], [[1, 3], [0, 2]], [[0, 2], [1, 3]]]


In [49]:
df

,Turntable,Gx(deg/h),Gy(deg/h),Gz(deg/h),Ax(m/s2),Ay(m/s2),Az(m/s2),GxTemp,GyTemp,GzTemp,AxTemp,AyTemp,AzTemp,Length
0,[0.0_0.0_0.0_0.0],-11.4340,9.5850,-1.5617,0.000568,9.801456,0.028688,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,36
1,[180.0_0.0_0.0_0.0],11.4505,9.5640,1.7026,0.001749,9.801449,0.034891,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,36
2,[180.0_180.0_0.0_0.0],-11.4230,-9.6324,1.7514,0.000791,-9.800910,-0.026932,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,36
3,[0.0_180.0_0.0_0.0],11.4517,-9.5893,-1.7858,0.001956,-9.800906,-0.032927,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,36
4,[90.0_0.0_0.0_0.0],-1.6037,9.5867,11.5125,-0.001966,9.801437,0.032302,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,36
5,[270.0_0.0_0.0_0.0],1.5837,9.6577,-11.2521,0.004385,9.801462,0.031391,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,36
6,[270.0_180.0_0.0_0.0],1.7144,-9.6477,11.4680,0.004306,-9.800772,-0.029327,-47.000000,-47.000000,-47.000000,-46.000000,-46.000000,-46.000000,36
7,[90.0_180.0_0.0_0.0],-1.7116,-9.6559,-11.5177,-0.001683,-9.800857,-0.030491,-47.000000,-47.000000,-47.000000,-46.000000,-46.000000,-46.000000,36
8,[0.0_90.0_0.0_0.0],-9.6110,-11.4727,-1.6228,-9.800020,-0.001665,0.060223,-47.000000,-47.000000,-47.000000,-46.000000,-46.000000,-46.000000,36
9,[180.0_90.0_0.0_0.0],9.6208,-11.4627,1.6753,9.802362,0.000042,-0.057838,-48.000000,-48.000000,-48.000000,-46.000000,-46.000000,-46.000000,36
